# Parallel teleport

Start by initializing the environment

In [ ]:
from qdk.qsharp import init, TargetProfile, compile, circuit
from qdk.simulation import AC1000, NoiseConfig
from qdk.widgets import Atoms, Circuit, Histogram

init(target_profile=TargetProfile.Base)

## Teleportation

This uses controlled operations rather than classical measurement to avoid mid-circuit measurement.

It is written to spread multiple instances of teleportation across the available qubits

In [ ]:
%%qsharp

operation Teleport(msg : Qubit, alice : Qubit, bob: Qubit) : Unit {
    // Create some entanglement that we can use to send our message.
    H(alice);
    CNOT(alice, bob);

    // Encode the message into the entangled pair.
    CNOT(msg, alice);
    H(msg);

    CNOT(alice, bob);
    Controlled Z([msg], bob);
}

operation Layout(instances: Int) : Result[] {
    // Partitions the teleport instances across rows and columns on the machine
    let cols = if instances >= 12 {36} else {instances * 3};
    let rows = (instances + 11) / 12;  // 1 to 12 = 1, 13 to 24 = 2, etc.

    use qubits = Qubit[instances * 3];
    mutable results: Result[] = [];

    for i in 0..instances-1 {
        let rowId = i / 12;
        let colId = (i % 12) * 3;
        let idx = colId + (rowId * 36);

        // Prep state on msg qubit
        if i % 4 == 1 {
            X(qubits[idx]);
        } elif i % 4 == 2 {
            H(qubits[idx]);
        }
        elif i % 4 == 3 {
            SX(qubits[idx]);
        }

        Teleport(qubits[idx], qubits[idx + 1], qubits[idx + 2]);

        // Reverse state prep on Bob's qubit
        if i % 4 == 1 {
            X(qubits[idx + 2]);
        } elif i % 4 == 2 {
            H(qubits[idx + 2]);
        }
        elif i % 4 == 3 {
            X(qubits[idx + 2]);
            SX(qubits[idx + 2]);
        }
        results += [MResetZ(qubits[idx + 2])];
        // let _ = MResetEachZ([qubits[idx], qubits[idx + 1]]);
        ResetAll([qubits[idx], qubits[idx + 1]]);
    }

    return results;
}


## Circuit

Verify the circuit for two rounds of teleportation

In [ ]:
Circuit(circuit("Layout(1)"))

## Visualization

Now run 120 rounds of teleportation in parallel and visualize

In [ ]:
# Compile the code to the generic QISA as usual
qisa = compile("Layout(120)")

# Get the target machine and compile for it
ac1000 = AC1000()
ops = ac1000.compile(qisa)

# Visualize the circuit on the hardware
Atoms(machine_layout = ac1000.get_layout(), trace_data = ac1000.get_trace(ops))

## Clifford simulation

Run simulations for 1,000 shots. First write a helper so the historgrams are readable

In [ ]:
# Helper to build 'correctness' histogram

from qsharp import Result

def expect_zeros_histogram(output):
    results = []

    for shot in output:
        has_loss = False
        has_one = False

        for result in shot:
            if result == Result.Loss:
                has_loss = True
            elif result == Result.One:
                has_one = True

        if has_loss and has_one:
            results.append("Flip & Loss")
        elif has_loss:
            results.append("Loss")
        elif has_one:
            results.append("Flip")
        else:
            results.append("Correct")

    return Histogram(results)


In [ ]:
results = ac1000.simulate(qisa, shots=1000)
expect_zeros_histogram(results)

In [ ]:
# Now with noise.
noise = NoiseConfig()
noise.sx.loss = 0.001
results = ac1000.simulate(qisa, shots=1000, noise=noise)
expect_zeros_histogram(results)